In [218]:
import json
import os
from tqdm import tqdm
import re
import numpy as np

#GPT4 explanations naive prompt (The model needs to do smth)
#test_path = 'data/answer/toollama_2_correct_prompt_timeout_fixed_gpt4_explanations_test/test'
#No explanations
#test_path = "data/answer/toollama_2_correct_prompt_timeout_fixed_test_no_explanations"
#Toolllama-2-v2
#test_path = "data/answer/toollama_2_v2_correct_prompt_timeout_fixed_explanations_test_v2"
#Toolllama-2-gpt4
#test_path= "data/answer/toollama_2_correct_prompt_timeout_fixed_gpt4_explanations_test_v3"
#Toolllama-2-v2 gpt-4
#test_path = "data/answer/toollama_2_correct_prompt_timeout_fixed_starling_explanations_test_v3"
#Toolllama-2-v2 Starling
#test_path = "data/answer/toollama_2_correct_prompt_timeout_fixed_llama-2-chat_explanations_test_v3"
#Toollama-reproduced with inner thoughts
#test_path = "data/answer/toolllama_7b_reproduction_test_trained_on_train_part_of_cleaned_train_checkpoint_600"
#Toolllama-post-finetuned on full val + corrections
#test_path = "data/answer/toollama_7B_post_finetuned_full_val_error_correction_test"
# Toolllama-post-finetuned on half val + corrections
#test_path = "data/answer/toollama_7B_post_finetuned_half_val_error_correction_test"
# Toolllama-post-finetuned on half train + corrections
# test_path = "data/answer/toollama_7B_post_finetuned_half_train_error_correction_test"
# Toolllam trained from scratch with error corrections
# test_path = "data/answer/toollama_7B_from_scratch_full_train_val_error_correction_test"
# Toolllam trained from scratch with error corrections v2
#test_path = "data/answer/toollama_7B_from_scratch_full_train_val_error_correction_test_v2"
# Toollama post finetuned on error corrections attempt 2 (model not trained on function calls)
# test_path = "data/answer/toollama_7B_post_finetuned_full_val_error_correction_test_attempt_2_12_06_2024"
# Toollama post finetuned on error corrections attempt 2 (model not trained on function calls)
test_path = "data/answer/toollama_7B_post_finetuned_full_val_error_correction_test_14_06_2024"

#ground_truth_test_path = 'data/self_correction/manually_created_multistep_res_dataset.jsonl'
# New test
ground_truth_test_path = "data/self_correction/dataset_gen/res_dataset_2/toolbench_multistep_dataset_2.json"

In [219]:
def detect_error(content: str):
    error_message = re.findall('{"error": "([\s\S]*)", "response"', content)
    assert len(error_message) <= 1
    if error_message:
        error_message = error_message[0]
    else:
        error_message = None
    if error_message:
        return True
    # проверка на ошибку с Incorrect
    if 'Incorrect' in content or 'does not exist' in content:
        return True
    # проверка на пустой респонс. если респонс пустой, то скорее всего распарсится json.loads
    try:
        json_response = json.loads(content)
        if not json_response['response']:
            return True
        pass
    except:
        pass
    return False

def list_api_called(item):
    trys = item['trys']
    assert len(trys) == 1
    chain = trys[0]['chain']
    res = [item['answer_generation']['query'], item['filename']]
    api_dict = dict()
    for message in chain:
        if message['node_type'] == 'Action':
            api_dict = dict()
            curr_api = message['description']
            api_dict['tool_name'] = curr_api
        if message['node_type'] == 'Action Input':
            try:
                api_dict['params'] = json.loads(message['description'])
            except:
                api_dict['params'] = {}
            api_dict['function'] = message['observation']
            res.append(api_dict)
        if message['node_type'] == 'Thought':
            api_dict['thought'] = message['description']
    return res

ground_truth = dict()
with open(ground_truth_test_path, 'r') as f:
    for i, line in enumerate(f):
        item = json.loads(line)
        ground_truth[item['user']] = item

responses = dict()
for filename in os.listdir(test_path):
    item = json.load(open(test_path + '/' + filename, 'r'))
    item['filename'] = filename
    responses[item['answer_generation']['query']] = item

responses_correct_format = dict()
for key in responses:
    responses_correct_format[key] = list_api_called(responses[key])

Посмотрим на то, вообще какие ошибки тут есть

In [220]:
for item in responses_correct_format.values():
    for step in item[2:]:
        if step['tool_name'] != 'Finish':
            if detect_error(step['function']):
                print(step['function'])

{"error": "Message error...", "response": "{'error': {'type': 'DataException', 'message': 'no data', 'code': 800}}"}
{"error": "Message error...", "response": "{'error': {'type': 'DataException', 'message': 'no data', 'code': 800}}"}
{"error": "Message error...", "response": "{'error': {'type': 'DataException', 'message': 'no data', 'code': 800}}"}
{"error": "Message error...", "response": "{'error': {'type': 'DataException', 'message': 'no data', 'code': 800}}"}
{"error": "Message error...", "response": "{'error': {'type': 'DataException', 'message': 'no data', 'code': 800}}"}
{"error": "Message error...", "response": "{'error': {'type': 'DataException', 'message': 'no data', 'code': 800}}"}
{"error": "", "response": "{'message': \"Endpoint '/bylocation/bycitystatecountry/Valdez/Alaska/United States/Turlock/California/United States' does not exist\"}"}
{"error": "", "response": "{'message': \"Endpoint '/bylocation/bycitystatecountry/Valdez/California/United States/Turlock/Alaska/Unite

Кажется, все задетектировали. Попробуем посмотреть на то, какие корректные цепочки есть

In [221]:
def return_correct_chain(item):
    correct_chain = item[:2]
    for step in item[2:]:
        if step['tool_name'] == 'Finish':
            if 'return_type' in step['params'] and step['params']['return_type'] == 'give_answer':
                correct_chain.append(step['tool_name'])
        else:
            if not detect_error(step['function']):
                correct_chain.append(step['tool_name'])
    return correct_chain


In [222]:
for key in responses_correct_format:
    print(responses_correct_format[key][0])
    print(return_correct_chain(responses_correct_format[key])[2:])
    print('#' * 100)

Do you have any details on the history of track 1872634217? Including its release date and any notable occurrences from that day?
['get_date_fact_for_numbers', 'Finish']
####################################################################################################
Is there a specific style or classification that best describes the music found on the album with ID 437584137?
[]
####################################################################################################
I'm having trouble converting an HSL color code to RGB. The code is 18.0, 23, 68.0. Can you walk me through the process and also convert me the red value from kilometers to meters? What's so special about the last number?
['hsl_to_rgb_for_convexity', 'Finish']
####################################################################################################
When it comes to the album with ID 387210487, could you classify its sound into a particular genre?
['album_for_deezer', 'Finish']
####################

Посмотрим на процент случаев, когда у модели получилось вызвать тот же самый тул после цепочки неправильных вызовов
И процент случаев, когда у модели получилось вызвать другой тул после цепочки неправильных вызовов

In [223]:
correct_same_tool_call_after_unsuccesful_steps = 0
correct_other_tool_call_after_unsuccessful_steps = 0
for item in responses_correct_format.values():
    flag = False
    last_tool_error = None
    for step in item[2:]:
        if step['tool_name'] != 'Finish':
            if detect_error(step['function']):
                if flag == False:
                    flag = True
                    last_tool_error = step['tool_name']
                else:
                    if step['tool_name'] != last_tool_error:
                        last_tool_error = step['tool_name']
            else:
                if flag == True:
                    if last_tool_error == step['tool_name']:
                        print('Same')
                        print(item)
                        print('#' * 100)
                        correct_same_tool_call_after_unsuccesful_steps += 1
                        flag = False
                        last_tool_error = None
                    else:
                        print('Other')
                        print(item)
                        print('#' * 100)
                        correct_other_tool_call_after_unsuccessful_steps += 1
                        flag = False
                        last_tool_error = None   
print(correct_same_tool_call_after_unsuccesful_steps)
print(correct_other_tool_call_after_unsuccessful_steps)

Other
['When it comes to the album with ID 387210487, could you classify its sound into a particular genre?', '56_CoT@1.json', {'tool_name': 'genre_for_deezer', 'params': {'is_id': '387210487'}, 'function': '{"error": "Message error...", "response": "{\'error\': {\'type\': \'DataException\', \'message\': \'no data\', \'code\': 800}}"}', 'thought': 'Tool call returned an error. To correct it I need to do the following. I need to call the "genre_for_deezer" function with the argument "is_id" set to "387210487". This will allow me to retrieve the genre information for the given album ID. By using this function, I can provide the user with the desired genre classification for the album they are interested in.'}, {'tool_name': 'genre_for_deezer', 'params': {'is_id': '387210487'}, 'function': '{"error": "Message error...", "response": "{\'error\': {\'type\': \'DataException\', \'message\': \'no data\', \'code\': 800}}"}', 'thought': 'Tool call returned an error. To correct it I need to do th

Это для тулламы с объяснением ошибок

Is there a specific style or classification that best describes the music found on the album with ID 161755?

When it comes to the album with ID 72512, could you classify its sound into a particular genre?

I'm planning a trip from Norwalk, Alaska to San Bernardino, California in the US. Can you tell me the distance between these two cities in miles and convert it to m for me?

Would you happen to know what category of music the album with ID 72512 falls under?

I'm interested in learning more about the album with ID 211939382. Is it possible for you to provide its genre?

Does the album with ID 162390622 adhere to a specific musical genre? If so, which one?

Could you please identify the genre of the album with the specified ID 6592547?

Can you tell me the musical style of the album with ID 527811?

I weigh 115.0 pounds and I'm 5.0 feet tall. What is my Body Mass Index?

What is the genre of album with id 326314867?'

Туллама без объяснения ошибок

'What is the genre of album with id 326314867?

Посчитаем процент кейсов с ошибками

In [224]:
cases_w_errors = 0
for item in responses_correct_format.values():
    correct_chain = item[:2]
    flag = False
    for step in item[2:]:
        if not step['tool_name'] == 'Finish':
            if detect_error(step['function']):
                flag = True
    if flag:
        cases_w_errors += 1
cases_w_errors

34

Выведем все вызовы последних тулов во всех случаях

In [225]:
ground_truth

{'How many albums does Shawty has? Please provide me an interesting information about obtained number.': {'user': 'How many albums does Shawty has? Please provide me an interesting information about obtained number.',
  'assistant1_tool_name': 'artist_for_deezer',
  'assistant1_params': {'artist': 'Shawty'},
  'assistant2_tool_name': 'get_math_fact_for_numbers',
  'function1': '{"id":1075733,"name":"Shawty","link":"https:\\/\\/www.deezer.com\\/artist\\/1075733","share":"https:\\/\\/www.deezer.com\\/artist\\/1075733?utm_source=deezer&utm_content=artist-1075733&utm_term=0_1701348321&utm_medium=web","picture":"https:\\/\\/api.deezer.com\\/artist\\/1075733\\/image","picture_small":"https:\\/\\/e-cdns-images.dzcdn.net\\/images\\/artist\\/a8bf32d9b3449bdff0e97ccf78788b07\\/56x56-000000-80-0-0.jpg","picture_medium":"https:\\/\\/e-cdns-images.dzcdn.net\\/images\\/artist\\/a8bf32d9b3449bdff0e97ccf78788b07\\/250x250-000000-80-0-0.jpg","picture_big":"https:\\/\\/e-cdns-images.dzcdn.net\\/images\\

In [226]:
def hardcoded_params_and_func_names(func_name, params_dict):
    if func_name == "m_one_unit_of_measure_to_another_for_measurement_units_converter":
        func_name = "convert_from_one_unit_of_measure_to_another_for_measurement_units_converter"
    if isinstance(params_dict, str):
        params_dict = json.loads(params_dict)
    new_params_dict = dict()
    for key, val in params_dict.items():
        if key == "is_id" and func_name == "album_for_deezer":
            key = "album_id"
        if key == "watchid" and func_name == "get_media_links_for_watch_database":
            key = "id"
        if isinstance(val, str):
            if val.isdigit():
                val = int(val)
        if isinstance(val, float):
            val = np.round(val, 2)
        new_params_dict[key] = val
    return func_name, new_params_dict

correct_tool_count = 0
correct_tool_and_params_count = 0
corr_ids = []
for id, key in enumerate(responses_correct_format.keys()):
    if responses_correct_format[key][-1]['tool_name'] != 'Finish':
        idx = -1
    else:
        idx = -2
    resp_tool_name = responses_correct_format[key][idx]['tool_name']
    resp_params = responses_correct_format[key][idx]['params']
    resp_tool_name, resp_params = hardcoded_params_and_func_names(resp_tool_name, resp_params)
    resp_function = responses_correct_format[key][idx]['function']
    # print(resp_tool_name, resp_params, resp_function)
    # print('#')
    if 'assistant3_tool_name' in ground_truth[key] and 'assistant3_params' in ground_truth[key] and 'function3' in ground_truth[key]:
        tool_name_key, params_key, func_key = 'assistant3_tool_name', 'assistant3_params', 'function3'
    elif 'assistant2_tool_name' in ground_truth[key] and 'assistant2_params' in ground_truth[key] and 'function2' in ground_truth[key]:
        tool_name_key, params_key, func_key = 'assistant2_tool_name', 'assistant2_params', 'function2'
    else:
        tool_name_key, params_key, func_key = 'assistant1_tool_name', 'assistant1_params', 'function1'
    gt_tool_name = ground_truth[key][tool_name_key]
    gt_params = ground_truth[key][params_key]
    gt_tool_name, gt_params = hardcoded_params_and_func_names(gt_tool_name, gt_params)
    gt_func = ground_truth[key][func_key]
    # print(gt_tool_name, gt_params, gt_func)

    if gt_tool_name == resp_tool_name:
        correct_tool_count += 1
        if gt_params == resp_params:
            correct_tool_and_params_count += 1
            corr_ids.append(id)
        else:
            print(id, '#' * 100)
            print(resp_tool_name, resp_params, resp_function)
            print('#')
            print(gt_tool_name, gt_params, gt_func)
            print('#' * 100)
    else:
        print(id, '#' * 100)
        print(resp_tool_name, resp_params, resp_function)
        print('#')
        print(gt_tool_name, gt_params, gt_func)
        print('#' * 100)

0 ####################################################################################################
get_date_fact_for_numbers {'month': 10, 'day': 20, 'fragment': 'true'} {"error": "", "response": "{'text': 'the Soviet Union performs the first armed test of a submarine-launched ballistic missile, launching an R-13 from a Golf class submarine', 'year': 1961, 'number': 294, 'found': True, 'type': 'date'}"}
#
get_date_fact_for_numbers {'month': 8, 'day': 24} {"error":"","response":"{'text': 'the Treaty of Córdoba is signed in Córdoba, now in Veracruz, Mexico, concluding the Mexican War of Independence from Spain', 'year': 1821, 'number': 237, 'found': True, 'type': 'date'}"}
####################################################################################################
1 ####################################################################################################
genre_for_deezer {'is_id': 437584137} {"error": "Message error...", "response": "{'error': {'type': 'DataExcepti

In [227]:
correct_tool_count

62

In [228]:
correct_tool_and_params_count

14

In [229]:
corr_ids

[3, 8, 10, 12, 13, 20, 35, 53, 54, 65, 73, 81, 82, 87]